# <font color='CC3D3D'> Feature Generation & Clansing2 Cluster
    
### 1. Feature Generation
- <span style="color:green">**Master_df.csv, user_spec.csv** </span>를 가지고 새로운 Feature 생성

### 2. Drop Feature
- ['loanapply_insert_time','user_id','insert_time', 'enter_year','enter_month','month','birth_year', 'month_day','연령대','소득수준',      '현재생활형편CSI_성별','현재생활형편CSI_나이', '현재생활형편CSI_소득']를 삭제

### 3. Deployment CSV
- 생성한 데이터를 포함한 <span style="color:blue">**Master_df_v2_cluster.csv** </span>를 생성

# Import

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns

# Data Load

In [2]:
tp_master = pd.read_csv('../Data/Master_df.csv')

In [3]:
user_spec = pd.read_csv('../Data/2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908/user_spec.csv')

# Feature Generation

In [4]:
# 1. feature engineering applicationID 안에서 loanapply_insert_time의 순위

def loanapply_insert_time_rank(master_df):    
    # 데이터 형태 변경
    master_df.loanapply_insert_time = master_df.loanapply_insert_time.astype('datetime64')
    master_df['loanapply_insert_time_rank'] = master_df.groupby('application_id')['loanapply_insert_time'].rank(method = 'min',ascending=False)
    
    return master_df

In [5]:
# 2. 한 applicationID 내에서 loan_limit 평균과 관측값의 차이 
def loan_limit_avg_sub(master_df): 
    tp_df =  master_df.groupby('application_id')['loan_limit'].mean().reset_index().rename(columns = {'loan_limit': 'loan_limit_avg'})
    master_df = pd.merge(master_df, tp_df, how = 'left')
    master_df['loan_limit_avg_sub'] = master_df['loan_limit'] - master_df['loan_limit_avg']
    master_df.drop('loan_limit_avg', axis = 1 ,inplace = True)
    return master_df

In [6]:
# 3. loan_rate 평균과 관측값의 차이
def loan_rate_avg_sub(master_df):
    tp_df =  master_df.groupby('application_id')['loan_rate'].mean().reset_index().rename(columns = {'loan_rate': 'loan_rate_avg'})
    master_df = pd.merge(master_df, tp_df, how = 'left')
    master_df['loan_rate_avg_sub'] = master_df['loan_rate'] - master_df['loan_rate_avg']
    master_df.drop('loan_rate_avg', axis = 1 ,inplace = True)
    return master_df

In [7]:
# 4. 한 applicationID bank_id의 다양성
def bankID_diversity(master_df):
    tp_df = master_df.groupby('application_id')['bank_id'].nunique().reset_index().rename(columns = {'bank_id': 'bank_diversity'})
    master_df = pd.merge(master_df, tp_df, how = 'left')
    return master_df

In [8]:
# 5. 한 applicationID product_id의 다양성
def productID_diversity(master_df):
    tp_df = master_df.groupby('application_id')['product_id'].nunique().reset_index().rename(columns = {'product_id': 'product_diversity'})
    master_df = pd.merge(master_df, tp_df, how = 'left')
    return master_df

In [9]:
tp_master = loanapply_insert_time_rank(tp_master)
tp_master = loan_limit_avg_sub(tp_master)
tp_master = loan_rate_avg_sub(tp_master)
tp_master = bankID_diversity(tp_master)
tp_master = productID_diversity(tp_master)

In [10]:
# 1. Application 당 결측치 개수
def missing_value_count(master_df):
    tp_df = pd.concat([user_spec['application_id'], pd.DataFrame(user_spec.isnull().sum(axis=1))], axis=1).rename(columns = {0:'missing_value_cnt'})
    master_df = pd.merge(master_df, tp_df, how='left', on='application_id')
    return master_df

In [11]:
# 2. 근속 연수
def working_year(master_df):
    current_year = 2022
    company_enter_year = master_df['enter_year']
    master_df['working_year'] = current_year - company_enter_year + 1
    return master_df

In [12]:
# 3. 근속 개월 수 
def working_month(master_df):
    current_year = 2022
    company_enter_year = master_df['enter_year']
    working_year = current_year - company_enter_year
    master_df['working_month'] = working_year * 12 + (6 - master_df['enter_month'] + 1)
    master_df['working_month'] = master_df['working_month'].apply(lambda x : 0 if x<0 else x)
    return master_df

In [13]:
# 4. 나이
def age(master_df):
    current_year = 2022
    birth = master_df.birth_year
    master_df['age'] = current_year - birth + 1
    return master_df

In [14]:
tp_master = missing_value_count(tp_master)
tp_master = working_year(tp_master)
tp_master = working_month(tp_master)
tp_master = age(tp_master)

In [15]:
# 공휴일
def holiday(master_df):
    holiday = ['2022-03-01', '2022-03-06', '2022-03-09', '2022-03-13', '2022-03-20', '2022-03-27',
               '2022-04-03', '2022-04-10', '2022-04-17', '2022-04-24',
               '2022-05-01', '2022-05-05', '2022-05-08', '2022-05-15', '2022-05-22', '2022-05-29']
    master_df['holiday'] = master_df['loanapply_insert_time'].apply(lambda x : 1 if str(x)[:10] in holiday else 0)
    return master_df

In [16]:
tp_master = holiday(tp_master)

In [17]:
tp_master['weekday'] = tp_master.loanapply_insert_time.dt.weekday 
tp_master['hour'] = tp_master.loanapply_insert_time.dt.hour # drop

In [18]:
# WeekDay
tp_master['weekday_sin'] = np.sin(tp_master.weekday*(2.*np.pi/7))
tp_master['weekday_cos'] = np.cos(tp_master.weekday*(2.*np.pi/7))
# Hour
tp_master['hour_sin'] = np.sin(tp_master.hour*(2.*np.pi/24))
tp_master['hour_cos'] = np.cos(tp_master.hour*(2.*np.pi/24))

In [19]:
# Purpose
tp_master['foreign'] = 0

tmp_dict = {'SWITCHLOAN':'대환대출', 
            'LIVING':'생활비',
            'INVEST':'투자',
            'ETC':'기타',
            'HOUSEDEPOSIT':'전월세보증금', 
            'BUSINESS':'사업자금', 
            'BUYHOUSE':'주택구입',
            'BUYCAR':'자동차구입'}
for key in tmp_dict:
    value = tmp_dict[key]
    tp_master.loc[tp_master['purpose']==key,'foreign'] = 1
    tp_master.loc[tp_master['purpose']==key,'purpose'] = value

In [20]:
tmp_purpose = pd.DataFrame(tp_master['purpose'].value_counts())
tmp_purpose = tmp_purpose.reset_index()
tmp_purpose = tmp_purpose.reset_index()
tmp_purpose.columns = ['rank','purpose','count']

In [21]:
tmp_purpose

,rank,purpose,count
0,0,생활비,6936238
1,1,대환대출,4106977
2,2,전월세보증금,585196
3,3,주택구입,580668
4,4,사업자금,560083
5,5,기타,407458
6,6,투자,289304
7,7,자동차구입,61326


In [22]:
tmp_dict = {'생활비':7, 
            '대환대출':6,
            '전월세보증금':5,
            '주택구입':4,
            '사업자금':3,
            '기타':2, 
            '투자':1, 
            '자동차구입':0}
for key in tmp_dict:
    value = tmp_dict[key]
    tp_master.loc[tp_master['purpose']==key,'purpose'] = value

In [23]:
tp_master['purpose'].value_counts()

7    6936238
6    4106977
5     585196
4     580668
3     560083
2     407458
1     289304
0      61326
Name: purpose, dtype: int64

In [24]:
target = tp_master['is_applied']
tp_master.drop(['is_applied'],axis=1,inplace=True)
tp_master['is_applied'] = target

In [25]:
tp_master.isna().sum().sum() #3257239

3257239

In [26]:
tp_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13527250 entries, 0 to 13527249
Data columns (total 68 columns):
 #   Column                               Dtype         
---  ------                               -----         
 0   application_id                       int64         
 1   loanapply_insert_time                datetime64[ns]
 2   bank_id                              int64         
 3   product_id                           int64         
 4   loan_limit                           float64       
 5   loan_rate                            float64       
 6   user_id                              float64       
 7   insert_time                          object        
 8   credit_score                         float64       
 9   yearly_income                        float64       
 10  income_type                          object        
 11  employment_type                      object        
 12  houseown_type                        object        
 13  desired_amount           

# Drop Data 1st

In [27]:
drop_columns = ['loanapply_insert_time','insert_time',
                'enter_year','enter_month','month','birth_year',
                'month_day','연령대','소득수준',
                '현재생활형편CSI_성별','현재생활형편CSI_나이',
                '현재생활형편CSI_소득']

In [28]:
tp_master.drop(drop_columns,axis=1,inplace=True)

In [29]:
tp_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13527250 entries, 0 to 13527249
Data columns (total 56 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   application_id                       int64  
 1   bank_id                              int64  
 2   product_id                           int64  
 3   loan_limit                           float64
 4   loan_rate                            float64
 5   user_id                              float64
 6   credit_score                         float64
 7   yearly_income                        float64
 8   income_type                          object 
 9   employment_type                      object 
 10  houseown_type                        object 
 11  desired_amount                       float64
 12  purpose                              object 
 13  personal_rehabilitation_yn           float64
 14  personal_rehabilitation_complete_yn  float64
 15  existing_loan_cnt             

In [30]:
tp_master.isna().sum()

application_id                               0
bank_id                                      0
product_id                                   0
loan_limit                                   0
loan_rate                                    0
user_id                                      0
credit_score                                 0
yearly_income                                0
income_type                                  0
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn                   0
personal_rehabilitation_complete_yn          0
existing_loan_cnt                            0
existing_loan_amt                            0
enter_birth                                  0
gender                                       0
k_lending_rate                               0
us_lending_rate                              0
btc_price    

# Data Check

In [31]:
tp_master.holiday.value_counts()

0    12505774
1     1021476
Name: holiday, dtype: int64

# Deployment CSV

In [32]:
tp_master.to_csv('../Data/Master_df_v2_cluster.csv',index=False)